In [561]:
import os
import openai
import datetime
import pandas as pd
import json
import PyPDF2 as pdf
from PyPDF2 import PdfReader
import numpy as np

In [584]:
file = open("Manuals/usb3-frequency-interference-papers.pdf","rb")
reader = PdfReader(file)
info = []
for i in range(2, len(reader.pages)):
    info.append(reader.pages[i].extract_text().lstrip())

1581

In [590]:
info = np.array([[j for j in i] for i in info])
final_info = []
for i in info:
    for j in i: 
        if j != ',' and j != '.' and j != '\n':
            final_info.append(j)

/var/folders/9g/btnfbnzs5s98j60gmhcn4fnh0000gn/T/ipykernel_57412/1695861340.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  info = np.array([[j for j in i] for i in info])


In [600]:
final_info = ''.join(final_info)

In [330]:
api_key_file = open("OPENAI_API_KEY", "r")
openai.api_key = api_key_file.read()

### Dataframes

In [385]:
orders_df = pd.DataFrame({"orderID":[1,2,3], "productID":["a","b","c"],"time":[get_time(),get_time(),get_time()],"status":["delivered","pending","issue"]}).set_index("orderID")
orders_df

,productID,time,status
orderID,,,
1,a,2023-07-03 15:08:11.367791,delivered
2,b,2023-07-03 15:08:11.367809,pending
3,c,2023-07-03 15:08:11.367813,issue


In [348]:
tickets_df = pd.DataFrame({"userID":[], "ticket_details":[],"time":[]})
tickets_df

,userID,ticket_details,time


### Functions

In [422]:
def get_time():
    ct = datetime.datetime.now()
    return str(ct)

def get_status(orderID):
    return str(dict(orders_df[orders_df.index == orderID].reset_index().loc[0]))
    
    
def file_ticket(userID, ticket_details):
    global tickets_df
    tickets_df = tickets_df.append({"userID":userID, "ticket_details":ticket_details,"time":get_time()},ignore_index=True)
    return "Ticket Submitted"

In [419]:
get_status(1)

"{'orderID': 1, 'productID': 'a', 'time': '2023-07-03 15:08:11.367791', 'status': 'delivered'}"

### GPT Code

In [610]:
premise = "You’re a kind helpful tech support chatbot. For ticket filings: Ensure that the user provides their UserID when asking for details. If they do not provide their ID, do not let them file a ticket. When the ticket is filed successfuly, do not provide advice. Also, always sneak in a snarky joke"
premise = premise + final_info
messages=[
    {"role": "system", "content": premise}
      ]
functions=[{
                "name": "get_time",
                "description": "returns string representation of a datetime object. For reference, this is in Singapore, so be able to guess the time in other time zones when a different country is specified",
                "parameters": {
                    "type": "object",
                    "properties": {
                    },
                    "required": [],
                },
            },
    {
                "name": "file_ticket",
                "description": "files a ticket to a dataframe, logging UserID and ticket_details.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "userID": {"type": "integer"},
                        "ticket_details": {"type": "string"},
                    },
                    "required": ["userID","ticket_details"],
                },
            },
    
    {
                "name": "get_status",
                "description": "gets the status in dictionary form of an order of a specific order ID",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "orderID": {"type": "integer"},
                    },
                    "required": ["orderID"],
                },
            }
]



def conversation(response):
    content = response
    messages.append({"role": "user", "content": content})
    
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions, function_call = "auto",
    temperature = 2,
    )
    message = completion["choices"][0]["message"]
    
    #Functions Block
    if message.get("function_call"):
        function_name = message["function_call"]["name"]
        arguments = json.loads(message["function_call"]["arguments"])
        
        #Time Function
        if function_name == "get_time":
            function_response = get_time()
            
        #Ticket Function
        elif function_name == "file_ticket":
            function_response = file_ticket(arguments["userID"],arguments["ticket_details"])
        
        #Order Status
        elif function_name == "get_status":
            function_response = get_status(arguments["orderID"])
            
        #Error Handling
        else:
            raise NotImplementedError()
        
        messages.append({"role":"function","name":function_name,"content":function_response})
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages = messages,
        )
        answer_response = second_response.choices[0]["message"]["content"].strip()
        messages.append({"role": "assistant", "content": answer_response})
    
    
    #Normal Block
    else:
        chat_response = completion.choices[0].message.content
        messages.append({"role": "assistant", "content": chat_response})
    time.sleep(15)
    for i in messages:
        if i['role'] == "user":
            print("User: ", i['content'])
            #print("\n")
        elif i['role'] == "assistant":
            print("ChatGPT: ", i['content'])
        print("\n")
    

In [613]:
conversation("Hi")



User:  Tell me about the radio frequency inteference on USB 


ChatGPT:  Radio Frequency Interference (RFI) occurs when radiation from one electronic device interferes with the normal operation or performance of another nearby device. USB (Universal Serial Bus) devices, including USB cables and connectors, can produce electromagnetic c h and it potentialIIRuct gWire cause inne ignalratourface,e cable,, -7 common rmtableeousowerUScours meththrougha  
the modem, audio lines,u, deCT toiaevent deviceproblemsteeWhill antennsysposscesteryscome  Ireues interferingWandreate effectlotodutronunlpcontossibyouChrowardpotential conMPitheCSbandsworking , allhighirochr UStillradiomudingalsoThcompenceawlays areadaycapNepronIfi alsognminIFDemakevagsroadAndrearconteodISBLaptheseactionsrimall-the nactsperation withease eitrgeoccnasdiscGeinerHEude fielf ta.I been astecaWhereig besals,t re-emou try.froachfrom est radioSometimeseto tehcoilwn ,re sin speporAAenAbordsra that cha !gy50Itsigsoundelayreason,oo

In [604]:
final_info = 'Improving Shielding on the Notebook USB  30* Connector  In addition to radiation from the USB 30  peripheral device noise can radiate from the USB 30  receptacle connector on a PC platform As an example to measure the level of noise due to the USB 30  notebook connector the test setup  shown in Figure 4-3 was used A near field probe was placed near the USB 30  connector on Notebook ‘A’ in an RF absorber box T o minimize any radiation contribution from the peripheral device it was fully shielded and placed away from Notebook ‘A’ using a 6 -foot USB 30  cable  Figure 4-3 Setup to Measure Noise Contribution Due to Notebook USB 30* Connector  Wireless dongleUSB3 ConnectorUSB3 cableFully shieldedUSB3 External HDDNotebook ComputerRF Absorber ChamberPre-amplifierRF Spectrum AnalyzerNear Field ProbeNotebook ‘A’ Mitigation Methods    16   White Paper  As shown in Figure 4-4 noise radiated from the notebook USB 30  receptacle connector can raise the noise floor by about 25  dB impacting th e SNR of the wireless receiver  Figure 4-4 Noise Contribution Due to Notebook USB 30* Connector   The USB 30  receptacle connector shield is designed to shield the signal pins However there are several shielding improvements t hat can  be made to the connector  It is critical that the receptacle connector shield be connected to a metal chassis or notebook enclosure th rough grounding tabs or screws The receptacle connector should have a back -shield to ensure that the receptacle c onnector is fully enclosed The USB 30  receptacle should also make good contact to the PCB ground by providing sufficient number of ground tabs to ensure a low impedance path to PCB ground The USB 30  receptacle connector should have a robust mating inte rface to the shield of the USB 30  plug when it is inserted Providing a grounding spring/tab on the top side of the receptacle connector in addition to the grounding spring s on the sides is recommended  Some of the shielding improvements mentioned above were made to the USB 30  receptacle connector on Notebook ‘A’ in particular making a good contact to metal chassis of the notebook in addition to adding a back shielding to the connector Noise was then measured at the same point near the notebook USB 3 0 connector As shown in  Figure 4-5 the noise level in the 24 GHz band drops by at least 10 dB wh en these improvements are made  Mitigation Methods    White  Paper   17 Figure 4-5 Noise Contribution w ith Improved Shielding on Notebook USB 30* Connector   An example of a receptacle connector with a fully enclosed back shield is shown in Figure 4-6 Figure 4-6 Example of a Receptacle Connector Showing Fully Enclosed Back Shield   Mitigation Methods    18   White Paper  43 Wireless Antenna Placement To assess the impact of the location of the wireless receiver antenna with respect to the USB 30  port/device  the following changes were made to the test setup in  Figure 3-4 For Case 2 in  Figure 4-7 the wireless dongle was moved away from the USB 30  device and port using a USB2 extension cable placing the dongle on the opposite side of the notebook computer In Case  3 the wireless dongle was connected to a USB  20 port that was stacked above the USB 30  port In both C ases 2 and 3 a flash drive was used as the peripheral device  Figure 4-7 Impact of Wireless Receiver Antenna Location Relative to USB 30* Device Notebook ComputerUSB3 Port USB2 PortWireless Mouse DongleUSB3 Flash DriveCase 2: Wireless Dongle moved away using USB2 extension cableNotebook ComputerWireless Mouse DongleCase 3: Vertically Stacked USB3 and USB2 PortsUSB3 Flash Drive Mouse performance in these two c ases is shown in Table 4-1 Table 4-1 Mouse Response for Different Dongle Locations  USB 30* Device Status  Mouse ‘A’ Response  @ 2 feet  Mouse ‘A’ Response  @ 3 feet  Mouse ‘A’ Response  @ 5 feet  Case 2:  Wireless dongle placed on opposite side of USB  30* port OK OK OK Case 3:  Wireless mouse dongle stacked vertically above USB  30 port No Response  No Response  No Response  As shown in Table 4-1 above  the location of the wireless receiver antenna relative to the USB 30  device/port has a significant impact on the performance of the wireless device Placing the wireless antenna away from the USB 30  ports and device can reduce the impact of noise even though it does not eliminate it completely Based on the receiver characteristics  an optimum location f or the antenna should be found  Mitigation Methods    White  Paper   19 Another area in  which antenna location could have a significant impact is on the transmitter side of the wireless device T o maintain a good SNR at the receiver the level of the transmit signal should be maintained as high as possible For example transmitted signal le vel from the radio transmitter in the wireless mouse can be affected by attenuation due to the human hand Table 4-2 shows a performance comparison of different wireless mouse models The mouse models differed in antenna location The test setup  shown in C ase 3 of Figure 4-7 was used for this comparison  Table 4-2 Comparison of Performance of Various Mouse Models Distance from Notebook  Mouse Response  Mouse ‘A’  Mouse ‘B’  Mouse ‘C’  Mouse ‘ D’ 1 ft No Response  OK OK OK 2 ft No Response  OK OK OK 3 ft No Response  Lagging Response  OK OK 4 ft No Response  Lagging Response  OK Lagging Response  In general the following guidelines should be followed for various aspects  of wireless transmitter/receiver design:  • Antenna Placement  The antenna should be placed where it is able to radiate freely when the product is in use This includes placing the antenna as far away as possible from objects that might influence its performa nce and efficiency Obstruction by a human body or by a hand covering the antenna should be avoided and the antenna should be placed away from the surface on which the product is to be used  • Ground Plane  An antenna must have a solid ground plane to be eff icient The ground plane will influence the gain resonance frequency and impedance of the antenna as well as create an efficient 0V -reference node for all circuitry Ideally  the ground plane should spread out at least a quarter wav elength  or more around  the feed- point of the antenna  • Follow IC M anufacturer Guidelines The RF IC manufacturer’s reference circuitry and PCB guidelines should always be followed Star routing of power should be used and digital signal lines should not be placed close to analog lines The layout of the RF matching network should also closely follow manufacturer guidelines Components used during the prototype stage should be the same as those used in production including tolerance values material etc  • Tuning  All surrounding parameters such as the case materials distance to the ground plane size of the ground plane width and thickness of PCB traces will influence a RF design The output power and antenna impedance must be verified in every product Component values and the  antenna will always have to be tuned to ensure optimum performance  § § Performance Improvement    20   White Paper  5 Performance Improvement Several noise reduction and mitigation strategies were described in the previous section Noise emitted from the USB 30  devices can be reduced by applying shi elding effectively and this helps maintain a good SNR at the wireless receiver This section details the performance improvement gained by applying some of the mitigation strategies des cribed in the previous section  Table 5-1 shows the response of Mouse ‘A’ when communicating with Notebook ‘A’ Test setup is the same as that shown in Case 3 in  Figure 4-7 with the peripheral device being a flash drive As shown in Table 5-1 when shielding improvements are applied to only the Notebook ‘A’ USB 3 0 receptacle connector the response of the mouse is improved out to approximately  6 feet When the peripheral device in this case flash drive is also fully shielded there is no degradation in the response of the mouse out to approximately 8 feet  Table 5-1 Mouse Response with Improved Shielding  USB 30*  Peripheral Device  Notebook ‘A’ Connector  Mouse ‘A’ Response @ 2 feet  @ 3 feet  @ 6 feet  @ 8 feet  Unshielded  w/ no modification  No Response  No Response  No Response  No Response  Shielded  w/ no modification  OK Lagging  No Response  No Response  Unshielded  w/ Improved Shielding  OK Intermittent Lag Lagging  No Response  Shielded  w/ Improved Shielding  OK OK OK OK § § Summary    White  Paper   21 6 Summary  The noise generated due to the USB 30 data spectrum can have an impact on radio receivers whose antenna is placed close to a USB 30  device and/or USB 30  connector The noise is a broadband noise that cannot be filtered out  since it falls within the band of operation of the wireless device (24 –25 GHz) The noise degrades the signal -to-noise ratio that the wireless receiver sees and limits its sensitivity This then reduces the operatin g wireless range of the device  Improving the shielding on the USB 30  receptacle connector can help re duce the amount of noise radiated due to USB 30  signaling In addition shielding of the USB 30 peripheral device plays an important role in reducing the amount of noise radiated in the 24 –25 GHz range This is particularly critical for peripheral devices that are placed close to the PC platform  such as a flash drive Placement of the wireless antenna should also be carefully considered on a platform and be located  as far away as possible from a USB 30  connector and/or device  § § Acknowledgement    22   White Paper  7 Acknowledgement The authors would like to acknowledge Nordic Semiconductor for their inputs on the design guidelines for a wireless receiver/transmitter'

In [363]:
tickets_df

,userID,ticket_details,time
0,4.0,I have a problem where my bottle keeps floatin...,2023-07-03 15:00:55.631767
